<a href="https://colab.research.google.com/github/aeau/MAU-AML-labs/blob/develop/2-language-models-lab/1-word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Word2Vec ## 

In this notebook we will go through the step by step creation of the Continouous Bag Of Words (CBOW).
CBOW is an embedded model that makes use of a "fake task" -> [within short window, predict the current word] to extract a vector that shows the relationship between words.

### Continuous Bag Of Words ###

Adapted from Robert Guthrie

In [35]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.linalg

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"

In [37]:
# CBOW is a window view; we are trying to infer the word in the middle.
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right

raw_text= """Long Short-Term Memory (LSTM) is a recurrent neural network (RNN) architecture that has been designed
to address the vanishing and exploding gradient problems of conventional RNNs. Unlike feedforward neural networks,
RNNs have cyclic connections making them powerful for modeling sequences. 
They have been successfully used for sequence labeling and sequence prediction tasks,
such as handwriting recognition, language modeling, phonetic labeling of acoustic frames. However, in contrast to the deep neural
networks, the use of RNNs in speech recognition has been limited to phone recognition in small scale tasks. 
In this paper, we present novel LSTM based RNN architectures which make more effective
use of model parameters to train acoustic models for large vocabulary speech recognition. 
We train and compare LSTM, RNN and DNN models at various numbers of parameters and configurations.
We show that LSTM models converge quickly and give state of the art speech recognition performance for relatively small sized models.""".split()

# By deriving a set from "raw_text", we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

# Basic Tokenizer
word_to_ix = {word: i for i, word in enumerate(vocab)}

print(len(raw_text))
print(vocab_size)

152
106


In [38]:
print(vocab)

{'models.', 'designed', 'relatively', 'quickly', 'limited', 'network', 'make', 'sequences.', 'RNN', 'parameters', 'the', 'prediction', 'based', 'model', 'deep', 'such', 'neural', 'Long', 'address', 'at', 'architectures', 'vocabulary', 'that', 'LSTM,', 'acoustic', 'has', 'frames.', 'In', 'feedforward', 'scale', 'conventional', 'handwriting', 'paper,', 'various', 'give', 'Unlike', 'novel', 'sequence', 'LSTM', 'They', 'networks,', 'speech', 'sized', 'is', 'more', 'large', 'phone', 'used', 'recognition,', 'train', 'tasks,', 'We', 'performance', 'Short-Term', 'small', 'which', 'and', 'vanishing', 'recurrent', 'have', 'a', 'them', 'language', 'modeling', 'converge', 'of', 'effective', 'recognition', 'for', 'gradient', 'connections', 'state', 'problems', 'phonetic', 'contrast', 'However,', 'use', '(LSTM)', 'DNN', 'configurations.', 'compare', 'architecture', 'in', 'as', 'modeling,', 'been', 'labeling', 'recognition.', 'models', 'art', 'tasks.', 'we', 'powerful', 'present', 'numbers', 'RNNs', 

In [39]:
# list out keys and values separately
key_list = list(word_to_ix.keys())
val_list = list(word_to_ix.values())

In [41]:
# Now lets create a "dataset"
data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = []
    for j in range(CONTEXT_SIZE, 0, -1):
        context.append(raw_text[i - j])

    for j in range(1, CONTEXT_SIZE + 1):
        context.append(raw_text[i + j])
        
    target = raw_text[i]
    data.append((context, target))
print(data[:5])


[(['Long', 'Short-Term', '(LSTM)', 'is'], 'Memory'), (['Short-Term', 'Memory', 'is', 'a'], '(LSTM)'), (['Memory', '(LSTM)', 'a', 'recurrent'], 'is'), (['(LSTM)', 'is', 'recurrent', 'neural'], 'a'), (['is', 'a', 'neural', 'network'], 'recurrent')]


### Create the CBOW Model (as we have seen already other ANN) ###

We have to extend from nn.Module as all the other ANN

In [43]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embed_dim, context, hidden_size):
        super(CBOW, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.linear = nn.Sequential(
            nn.Linear(context*embed_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, vocab_size),
            nn.LogSoftmax(dim = -1)
        )
        
    def forward(self, inputs):
#         print(inputs.shape)
#         print(inputs)
        out = self.embedding(inputs)
#         print(out.shape)
        out = out.view(1, -1)
#         print(out.shape)
        out = self.linear(out)
#         print(out.shape)
        return out
    
    # This is what we are actually interested on
    def get_word_vector(self, word):
        out = self.embedding(word)
        return out


#### Lets break it down! ####

In [49]:
VOCAB_SIZE = len(vocab)
EMBEDD_DIM = 10
BATCH_SIZE = 6
FULL_CONTEXT_SIZE = CONTEXT_SIZE * 2
HIDDEN_SIZE = 256

example_tensor = torch.randint(0, VOCAB_SIZE, [BATCH_SIZE, FULL_CONTEXT_SIZE])
print(example_tensor)

tensor([[ 14,  10,  79,  60],
        [  5,  51, 100,  38],
        [ 53,   7,  99,  99],
        [ 81,  26,  83,  16],
        [ 56,  87,  77,   8],
        [ 25,  26,   9,  40]])


In [50]:
CBOW_embedding = nn.Embedding(VOCAB_SIZE, EMBEDD_DIM)
print(example_tensor.shape)
example_result = CBOW_embedding(example_tensor)
# Now we have a representation of the words in a vector of EMBEDD_DIM Dimensions
print(example_result.shape)
# example_result = torch.flatten(example_result, start_dim=1)
example_result = example_result.view(BATCH_SIZE, -1)
print(example_result.shape)

torch.Size([6, 4])
torch.Size([6, 4, 10])
torch.Size([6, 40])


In [51]:
print("input shape: ", EMBEDD_DIM * FULL_CONTEXT_SIZE)
print("output shape: ", HIDDEN_SIZE)
CBOW_hidden = nn.Linear(EMBEDD_DIM * FULL_CONTEXT_SIZE, HIDDEN_SIZE)
CBOW_hidden_relu = nn.ReLU()
example_result = CBOW_hidden(example_result)
example_result = CBOW_hidden_relu(example_result)
print(example_result.shape)

input shape:  40
output shape:  256
torch.Size([6, 256])


In [52]:
CBOW_output = nn.Linear(HIDDEN_SIZE, VOCAB_SIZE)
CBOW_output_soft = nn.LogSoftmax(dim = -1)
example_result = CBOW_output(example_result)
example_result = CBOW_output_soft(example_result)
print(example_result.shape)

torch.Size([6, 106])


In [53]:
print(example_result[0].argmax(-1))
print(key_list[val_list.index(example_result[0].argmax(-1))])
print(example_result[0])
print(example_result[1].argmax(-1))
print(key_list[val_list.index(example_result[1].argmax(-1))])
print(example_result[1])
# print(example_result[2].argmax(-1))
# print(example_result[3].argmax(-1))
# print(example_result[4].argmax(-1))

tensor(72)
problems
tensor([-4.8033, -4.1789, -4.7501, -4.4437, -4.9349, -4.4947, -4.5962, -4.6865,
        -4.2706, -5.0489, -4.7150, -4.2689, -4.6101, -5.0909, -4.4317, -4.7043,
        -4.6071, -4.2357, -4.1776, -4.7040, -4.6179, -4.6839, -4.6393, -5.0006,
        -4.8680, -5.0305, -4.7531, -4.9513, -4.5972, -4.5174, -4.9613, -4.6879,
        -4.8597, -4.8316, -4.7603, -4.8704, -4.5004, -4.6067, -4.7328, -4.6842,
        -4.6653, -4.7083, -4.8235, -4.8406, -4.7145, -4.7999, -4.7908, -4.7445,
        -4.5184, -4.6010, -4.5165, -4.5942, -4.3015, -4.5594, -5.0058, -4.4460,
        -4.9035, -4.6203, -4.7098, -4.5688, -4.6414, -5.0183, -4.7159, -4.3759,
        -4.8426, -4.7740, -4.5387, -4.7615, -4.5280, -4.9196, -4.9910, -4.6441,
        -4.1577, -4.3915, -5.4968, -4.6118, -4.5293, -4.3066, -4.8217, -4.9757,
        -4.3309, -4.7029, -4.2933, -4.7853, -4.5820, -4.9092, -4.3813, -4.4539,
        -4.9692, -4.8581, -4.8920, -4.7891, -5.0422, -4.6386, -4.3683, -4.7666,
        -4.5939, -4.

## Back to the notebook ##

In [54]:
# Simple helper method to transform the context to the expected int vector - tensor

def make_context_vector(context, word_to_ix, debug=False):
    if debug:
      print(context)
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

make_context_vector(data[0][0], word_to_ix, debug=True)

['Long', 'Short-Term', '(LSTM)', 'is']


tensor([17, 53, 77, 43])

In [55]:
def train(model, epochs, data, optimizer, loss_fn):
    model.train()
    losses = []
    for epoch in range(epochs):
        total_loss = 0
        for context, target in data:

            # Prepare inputs and targets 
            context_idxs = make_context_vector(context, word_to_ix)
            context_idxs = context_idxs.to(device)
            target_id = make_context_vector([target], word_to_ix)
            target_id = target_id.to(device)

            # Do not accumulate 
            model.zero_grad()

            # Step 3. Run the forward pass
            log_probs = model(context_idxs)
    #         break

            # Step 4. Compute your loss function.
            loss = loss_fn(log_probs, target_id)

    #         loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

            # Step 5. Do the backward pass and update the gradient
            loss.backward()
            optimizer.step()

            # Get the Python number from a 1-element Tensor by calling tensor.item()
            total_loss += loss.item()
        losses.append(total_loss)
    return losses
    

In [56]:
VOCAB_SIZE = len(vocab)
EMBEDD_DIM = 10
BATCH_SIZE = 6
FULL_CONTEXT_SIZE = CONTEXT_SIZE * 2
HIDDEN_SIZE = 256

loss_function = nn.NLLLoss() # Because we are using Log_softmax
model = CBOW(vocab_size, EMBEDD_DIM, FULL_CONTEXT_SIZE, HIDDEN_SIZE)
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

losses = train(model, 100, data, optimizer, loss_function)
model.eval()

print(losses)  # The loss decreased every iteration over the training data!

[695.8721408843994, 688.4224910736084, 681.0870337486267, 673.8598070144653, 666.7323791980743, 659.6977827548981, 652.7519869804382, 645.8902723789215, 639.1074109077454, 632.3980779647827, 625.7590072154999, 619.1865000724792, 612.6766057014465, 606.224250793457, 599.8256285190582, 593.4723887443542, 587.1625649929047, 580.8921282291412, 574.6608099937439, 568.4589643478394, 562.2875165939331, 556.1384534835815, 550.0095571279526, 543.8945552110672, 537.7902361154556, 531.6990878582001, 525.6131232976913, 519.5326136350632, 513.4545558691025, 507.3737608194351, 501.29139947891235, 495.20449340343475, 489.1113768815994, 483.0120348930359, 476.90717166662216, 470.79561322927475, 464.67776387929916, 458.5497826933861, 452.4149109721184, 446.2718482017517, 440.12113070487976, 433.9601615667343, 427.79011195898056, 421.6097915172577, 415.4242233633995, 409.2307227253914, 403.03252267837524, 396.83162090182304, 390.6302180290222, 384.42394638061523, 378.2179191708565, 372.016351044178, 365

In [57]:
# list out keys and values separately
key_list = list(word_to_ix.keys())
val_list = list(word_to_ix.values())

In [58]:
def similarity_cbow(word_1, word_2):
    
    # test word similarity
    print(word_1)
    print(word_2)
    w1_id = torch.tensor(word_to_ix[word_1], dtype=torch.long)
    w2_id = torch.tensor(word_to_ix[word_2], dtype=torch.long)
    w1_id = w1_id.to(device)
    w2_id = w2_id.to(device)
    
    word_1_vec = model.get_word_vector(w1_id)
    word_2_vec = model.get_word_vector(w2_id)
    
    # The norm of a vector (1D-matrix) is the square root of the sum of all the squared values within the vector.
    print(math.sqrt(torch.square(word_1_vec).sum()))    
    print(torch.linalg.norm(word_1_vec))
    print(torch.linalg.norm(word_2_vec))
    print(word_1_vec.dot(word_2_vec))
    
    word_distance = torch.linalg.norm(word_1_vec - word_2_vec)
    print("Distance between '{}' & '{}' : {:0.4f}".format(word_1, word_2, word_distance))
    word_similarity = (word_1_vec.dot(word_2_vec) / (torch.linalg.norm(word_1_vec) * torch.linalg.norm(word_2_vec)))
    print("Similarity between '{}' & '{}' : {:0.4f}".format(word_1, word_2, word_similarity))


In [59]:
similarity_cbow("neural", "network")

neural
network
3.502922064478045
tensor(3.5029, grad_fn=<CopyBackwards>)
tensor(3.2616, grad_fn=<CopyBackwards>)
tensor(3.8322, grad_fn=<DotBackward0>)
Distance between 'neural' & 'network' : 3.9044
Similarity between 'neural' & 'network' : 0.3354


In [60]:
def predict_middle_word(prev_words, post_words):
    prev_words = prev_words.split()
    post_words = post_words.split()

    input_words= make_context_vector(prev_words + post_words, word_to_ix)
    input_words = input_words.to(device)
    output = model(input_words)
    out_ind = output.argmax(1)
#     print(word_to_ix)
#     out_word = word_to_ix.itos[out_ind.item()]
    out_word = key_list[val_list.index(out_ind.item())]
    print(prev_words, out_word, post_words)

In [61]:
predict_middle_word("a recurrent", "network is")
predict_middle_word("LSTM is", "recurrent neural")

['a', 'recurrent'] neural ['network', 'is']
['LSTM', 'is'] a ['recurrent', 'neural']


### Now that you saw how to create the CBOW model (word2vec), you should work on doing the "opposite" model, Skip-Gram ###

Skip-gram as you saw on the lectures, reverses the problem so you need to predict through the "fake task" the context of the input